In [1]:
import sys
import os
from pathlib import Path

import sys
import os
from pathlib import Path
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
sys.path.append("../models")
processed_series_path = Path(parent_dir) / 'processed_series'
import copy
from scipy.special import erf
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
import pickle

print(sys.path)
from ICL import MultiResolutionPDF



['/home/admin-quad/Toni/LLMICL_inPCA/figures_analysis', '/home/admin-quad/anaconda3/envs/LLM/lib/python311.zip', '/home/admin-quad/anaconda3/envs/LLM/lib/python3.11', '/home/admin-quad/anaconda3/envs/LLM/lib/python3.11/lib-dynload', '', '/home/admin-quad/anaconda3/envs/LLM/lib/python3.11/site-packages', '/home/admin-quad/LLM/llama', '/home/admin-quad/Toni/LLMICL_inPCA', '../models']


In [2]:
all_files = sorted([path for path in processed_series_path.iterdir()], key=lambda x: x.name)
for index, file in enumerate(all_files):
    series_dict = pickle.load(file.open('rb'))
    llama_size = series_dict['llama_size']
    random_seed = series_dict['random_seed']
    kernel = series_dict.get('kernel', '')
    print(f"[{index}]: {file.name}, {llama_size}, {random_seed}, {kernel}")

[0]: uncorrelated_gaussian_centered_sigma_0.1_0.pkl, 13b, 2, 
[1]: uncorrelated_gaussian_centered_sigma_0.1_1.pkl, 13b, 3, 
[2]: uncorrelated_gaussian_centered_sigma_0.1_2.pkl, 7b, 2, 
[3]: uncorrelated_gaussian_centered_sigma_0.1_3.pkl, 7b, 3, 
[4]: uncorrelated_gaussian_centered_sigma_0.1_4.pkl, 70b, 2, 
[5]: uncorrelated_gaussian_centered_sigma_0.1_5.pkl, 70b, 3, 
[6]: uncorrelated_gaussian_centered_sigma_0.1_KDE_0.pkl, KDE, 2, gaussian
[7]: uncorrelated_gaussian_centered_sigma_0.1_KDE_1.pkl, KDE, 3, gaussian
[8]: uncorrelated_gaussian_centered_sigma_0.1_KDE_10.pkl, KDE, 2, cosine
[9]: uncorrelated_gaussian_centered_sigma_0.1_KDE_11.pkl, KDE, 3, cosine
[10]: uncorrelated_gaussian_centered_sigma_0.1_KDE_2.pkl, KDE, 2, epanechnikov
[11]: uncorrelated_gaussian_centered_sigma_0.1_KDE_3.pkl, KDE, 3, epanechnikov
[12]: uncorrelated_gaussian_centered_sigma_0.1_KDE_4.pkl, KDE, 2, tophat
[13]: uncorrelated_gaussian_centered_sigma_0.1_KDE_5.pkl, KDE, 3, tophat
[14]: uncorrelated_gaussian_cent

In [13]:

alpha = 1


file = all_files[24]
file = all_files[26]
file = all_files[30]
file = all_files[58]
file = all_files[56]

dist_type = 'gaussian'



processed_dict = pickle.load(file.open('rb'))
full_series = processed_dict['full_series']
rescaled_true_mean_arr = processed_dict.get('rescaled_true_mean_arr',None)
rescaled_true_sigma_arr = processed_dict.get('rescaled_true_sigma_arr',None)
true_PDF = processed_dict.get('true_PDF',None)
llama_size = processed_dict['llama_size']
mode = processed_dict['mode']
refine_depth = processed_dict['refine_depth']
random_seed = processed_dict['random_seed']
PDF_list = processed_dict['PDF_list']
time_series = processed_dict['time_series']
prec = processed_dict['prec']
kernel = processed_dict.get('kernel', '')
print(f"seed: {processed_dict['random_seed']}")
print(f"model name: {llama_size}")
print(f"mode: {mode}")
print(f"refine_depth: {refine_depth}")
print(f"prec: {prec}")
print(f"kernel: {kernel}")
print(f"name: {file.stem}")

seed: 1
model name: 70b
mode: all
refine_depth: 1
prec: 2
kernel: NA
name: uncorrelated_random_PDF_l_0.1_70b_1


In [14]:
### tune temperature 

for PDF in PDF_list:
    if alpha != 1:
        PDF.rescale_temperature(alpha)

In [15]:
### Calculate true discretized distribution: PDF_list_true
### Applies to gaussian distribution only

PDF_true_list = copy.deepcopy(PDF_list)
if true_PDF is not None:
    for i in range(len(PDF_true_list)):
        PDF_true_list[i] = true_PDF
else:
    for i, (PDF, PDF_true, true_mean, true_sigma) in enumerate(zip(PDF_list, PDF_true_list, rescaled_true_mean_arr, rescaled_true_sigma_arr)):
        def cdf(x):
            if dist_type == "gaussian":
                return 0.5 * (1 + erf((x - true_mean) / (true_sigma * np.sqrt(2))))
            elif dist_type == "uniform":
                lower_bound = true_mean - true_sigma
                upper_bound = true_mean + true_sigma
                return np.where(
                    (x >= lower_bound) & (x <= upper_bound),
                    (x - lower_bound) / (2 * true_sigma),
                    np.where(x < lower_bound, 0, 1)
                )
            else:
                raise NotImplementedError("This distribution type is not implemented.")
        
        PDF_true.discretize(cdf, mode="cdf")
        PDF_true.compute_stats()
        PDF_true_list[i] = PDF_true

In [17]:
from ipywidgets import interact
import matplotlib.pyplot as plt
from scipy.stats import norm, uniform
save_path_wild = f"../figures/animation/frame"
### Load multiple digits to MultiResolutionPDF
comma_locations = np.sort(np.where(np.array(list(full_series)) == ',')[0])
time_series_rescaled = (time_series-time_series.min()) / (time_series.max()-time_series.min()) * (8.5-1.5) + 1.5

plot1_log_scale = 0
log_scale = 0
truth_PDF = 1

### Plot distribution before ith comma
def digiprob_plotter(comma_idx=1, save_plot = False):
    if comma_idx == 0:
        start_idx = 0
    else:
        start_idx = comma_locations[comma_idx-1]+1
    if truth_PDF:
        fig, axs = plt.subplots(3, 1, figsize=(8, 5), dpi = 200)
    else:
        fig, axs = plt.subplots(2, 1, figsize=(8, 10/3), dpi = 200)
    # Adjust the horizontal space between subplots
    plt.subplots_adjust(hspace=0.7)
    # Plot the full array with a marker on the selected value

    time_series_plot = time_series_rescaled
    axs[0].plot(time_series_plot[:-1], marker='o', color='black', markersize=2, lw = '0.1', label = "observation")
    axs[0].scatter(comma_idx, time_series_plot[comma_idx], color='r', marker='o', label = "prediction")
    axs[0].set_xlabel('context length')
    axs[0].set_ylabel(r'rescaled $x$')
    
    if plot1_log_scale:
        axs[0].set_yscale('log')
        
    # Plot softmax distributions for each digit
    axs[1].set_ylabel("probability density")
    # axs[1].set_title(full_series[start_idx-30:start_idx] + "?")      
    PDF_list[comma_idx].compute_stats()
    PDF_list[comma_idx].plot(ax = axs[1], log_scale=log_scale, statistic = False)
    if truth_PDF:
        PDF_true_list[comma_idx].plot(ax = axs[2], log_scale=log_scale, statistic = False)
        axs[2].set_title('discretized truth')
        # axs[2].set_xlabel("Digit")
        axs[2].set_xlim(0,10)
    axs[1].set_xlim(0,10)
    # axs[1].set_ylim(0,5)
    axs[1].set_title('estimated density')
    axs[0].legend()
    axs[1].legend()
    if log_scale:
        axs[1].set_ylim(0.005,10)
        if truth_PDF:
            axs[2].set_ylim(0.0005,10)
    if save_plot: 
        plt.tight_layout()
        save_path = save_path_wild + f"{comma_idx}.png"
        plt.savefig(save_path)
        filenames.append(save_path)

# Adjust the range of the slider to match the number of commas in the series
num_commas = full_series.count(',')
interact(digiprob_plotter, comma_idx=(1, num_commas-1, 1))

interactive(children=(IntSlider(value=1, description='comma_idx', max=399, min=1), Checkbox(value=False, descr…

<function __main__.digiprob_plotter(comma_idx=1, save_plot=False)>

### Animate

In [18]:
# snapshots = np.linspace(1, 381, 38, endpoint=False, dtype=int)
snapshots = np.append(np.linspace(1, 21, 20, endpoint=False, dtype=int), np.linspace(31, 331, 15, endpoint=False, dtype=int))
snapshots 

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  31,  51,  71,  91, 111, 131,
       151, 171, 191, 211, 231, 251, 271, 291, 311])

In [19]:
filenames = []
for snapshot in snapshots:
    interact(digiprob_plotter, comma_idx=snapshot, save_plot = True)

interactive(children=(IntSlider(value=1, description='comma_idx', max=3, min=-1), Checkbox(value=True, descrip…

interactive(children=(IntSlider(value=2, description='comma_idx', max=6, min=-2), Checkbox(value=True, descrip…

interactive(children=(IntSlider(value=3, description='comma_idx', max=9, min=-3), Checkbox(value=True, descrip…

interactive(children=(IntSlider(value=4, description='comma_idx', max=12, min=-4), Checkbox(value=True, descri…

interactive(children=(IntSlider(value=5, description='comma_idx', max=15, min=-5), Checkbox(value=True, descri…

...

interactive(children=(IntSlider(value=231, description='comma_idx', max=693, min=-231), Checkbox(value=True, d…

interactive(children=(IntSlider(value=251, description='comma_idx', max=753, min=-251), Checkbox(value=True, d…

interactive(children=(IntSlider(value=271, description='comma_idx', max=813, min=-271), Checkbox(value=True, d…

interactive(children=(IntSlider(value=291, description='comma_idx', max=873, min=-291), Checkbox(value=True, d…

interactive(children=(IntSlider(value=311, description='comma_idx', max=933, min=-311), Checkbox(value=True, d…

In [20]:
# Create a GIF using snapshot made
import imageio.v2 as imageio

with imageio.get_writer('../figures/randomPDF_70B_snapshots.gif', mode='I', duration=500, loop=0) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
    for filename in filenames:
        os.remove(filename)